In [1]:
import subprocess
import numpy as np
import healpy as hp
import pickle
from random import randint
import joblib
import sklearn
import sys
import importlib

sys.path.append('../methods_code_Nresol/')
sys.path.append('../utils/')
from utils_circpatch import *
from methods2 import *
from vis_utils import *
import astro_cuts
importlib.reload(astro_cuts)

<module 'astro_cuts' from '/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/astro_cuts.py'>

In [2]:
fullskycoords = get_testbed_dict('FullSky_Bayestar')

In [3]:
numtiles = np.unique(get_largepix_for_smallpix(fullskycoords['coords'], 2048, 32))

In [4]:
len(numtiles), len(numtiles)/2368

(9447, 3.9894425675675675)

In [5]:
b17map = get_bayestar2017_map()
np.isnan(b17map[fullskycoords['coords']]).sum()

0

In [6]:
runname = 'fullsky_runs/16a_fwhm-6-1_'
wisemodel = joblib.load('spectra_matched/models/svm_panstarrs_quasarsep_balanced.joblib')
smodel = joblib.load('spectra_matched/models/svm_sdss_quasarsep_unbalanced.joblib')

cuts_list = [('distmod_median_cut', {'mindm': 8}), ('dm_sigma_cut', {'maxsig': 1.5}), 
             ('wise_svmnondetectioncombinedcut', {'model': wisemodel}), ('parallax_nan_edr3',),
             ('sdss_uvcut', {'model': smodel}), ('bayestar_chisq', {'maxchisq': 3.0})]
outer_kwargs = {'Nsidetile': 32, 'radius_deg_extra': 25/60, 'Nsideresol': 2048,
               'stars_presaved': '/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/data/lsdraw/stars_edr3/{}.fits'}
recon_kwargs = {'length': np.deg2rad(6.1/(2.355*60)), 'scale': 1.0, 'no_invvar': False}
kwargs_dict = {'outer_kwargs': outer_kwargs, 'recon_kwargs': recon_kwargs}
recon_func_name = 'gaussiankernel_reconstruction_effective_spherical3_blocks'

In [7]:
recondict = gnupartilewise_new(fullskycoords['coords'], recon_func_name, cuts_list, kwargs_dict, 
                               runname, sleep=7500, mem=4000, compile_pickle=True)

Using Star Directory: /n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/data/lsdraw/stars_edr3/{}.fits
Ckp
Missing / Corrupted files:  [3142, 3143, 3144, 3146, 3260, 3269, 3270, 3271, 3272, 3273, 3274, 3275, 3385, 3386, 3387, 3388, 3389, 3390, 3391, 3392, 3393, 3394, 3395, 3397, 3398, 3399, 3400, 3401, 3402, 3403, 3404, 3405, 3406, 3407, 3408, 3511, 3512, 3513, 3514, 3515, 3516, 3517, 3518, 3519, 3520, 3521, 3522, 3523, 3524, 3525, 3526, 3527, 3528, 3529, 3530, 3531, 3532, 3533, 3534, 3535, 3536, 3537, 3639, 3640, 3641, 3642, 3643, 3644, 3645, 3646, 3647, 3648, 3649, 3650, 3651, 3652, 3653, 3654, 3655, 3656, 3657, 3658, 3659, 3660, 3661, 3662, 3663, 3664, 3665, 3666, 3668, 3769, 3770, 3771, 3772, 3773, 3774, 3775, 3776, 3777, 3778, 3779, 3780, 3781, 3782, 3783, 3784, 3785, 3786, 3787, 3788, 3789, 3790, 3791, 3792, 3793, 3794, 3795, 3796, 3797, 3798, 3897, 3898, 3899, 3900, 3901, 3902, 3903, 3904, 3905, 3906, 3907, 3908, 3909, 3910, 3911, 3912, 3913, 3914, 3915, 3916, 3917, 3918, 3919, 

TypeError: cannot unpack non-iterable NoneType object

In [9]:
missing_tiles = []
for tile in numtiles:
    if not os.path.exists(runname+'_tmpdir/recon_{}.hdf5'.format(tile)):
        missing_tiles.append(tile)

In [ ]:
gnupartilewise_new(fullskycoords['coords'], recon_func_name, cuts_list, kwargs_dict, 
                               runname, sleep=3000, mem=8000, compile_pickle=True)

Dir already exists
Tiles file already exists
Only reconstructing for missing 2669 tiles
Using Star Directory: /n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/data/lsdraw/stars_edr3/{}.fits
Tmp pickle already exists


In [15]:
missing_tiles = []
for tile in numtiles:
    if not os.path.exists(runname+'_tmpdir/recon_{}.hdf5'.format(tile)):
        missing_tiles.append(tile)

In [16]:
len(missing_tiles)

1466

In [ ]:
recondict = gnupartilewise_new(fullskycoords['coords'], recon_func_name, cuts_list, kwargs_dict, 
                               runname, sleep=2000, mem=16000, compile_pickle=True)

Dir already exists
Tiles file already exists
Only reconstructing for missing 1466 tiles
Using Star Directory: /n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/data/lsdraw/stars_edr3/{}.fits
Tmp pickle already exists


In [30]:
missing_tiles = []
for tile in numtiles:
    if not os.path.exists(runname+'_tmpdir/recon_{}.hdf5'.format(tile)):
        missing_tiles.append(tile)
len(missing_tiles)

784

In [37]:
missing_coords = []
for t32 in missing_tiles:
    missing_coords.append(get_smallpix_in_tilepix(outer_kwargs['Nsidetile'], t32, 2048))
missing_coords = np.hstack(missing_coords)

In [43]:
np.all(np.unique(get_largepix_for_smallpix(missing_coords, 2048, outer_kwargs['Nsidetile'])) == missing_tiles)

True

In [ ]:
looptilewise(missing_coords, recon_func_name, cuts_list, kwargs_dict, runname, save_tilewise=True, compile_pickle=False)

Looping over 784 tiles
Using dir  /n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/data/lsdraw/stars_edr3/{}.fits
Region for tile4545
Cut0 990532 0.9971320168193272
Cut1 968233 0.9746844362837622
Cut2 975556 0.9820562301876118
Cut3 887953 0.8938695223685574
Cut4 993381 1.0
Cut5 670199 0.6746646050206315
Cuts_All 615997 0.6201014515075284
Tile 4545, NumPixels=4096, NumStars=615997, Star/Pix ratio = 150.389892578125
SigRef 0.1822883907996713
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 180, 16%ile = 265.7, median stars = 304.0, 84%ile = 346.0, max stars = 403
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 141, 16%ile = 240.0, median stars = 276.0, 84%ile = 311.0, max stars = 366
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 204, 16%ile = 268.0, median stars = 292.0, 84%ile = 315.0, max stars = 362
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) 

/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:51: RuntimeWarning: invalid value encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log10(np.array(dat['sdss_dr14_starsweep.psfflux'])[:, ib]), 0.0, np.inf)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:52: RuntimeWarning: divide by zero encountered in true_divide
  df['sdss.pmag_err_'+b] = (2.5/np.log(10)) * (sdss_flux_sig[:, ib]/np.array(dat['sdss_dr14_starsweep.psfflux'])[:, ib])


Cut0 851040 0.9965724670859647
Cut1 822040 0.9626133094135956
Cut2 829834 0.971740125789404
Cut3 754433 0.8834451448358075
Cut4 853962 0.9999941449728151
Cut5 613462 0.7183673373795475
Cuts_All 535600 0.6271905120455474
Tile 4546, NumPixels=4096, NumStars=535600, Star/Pix ratio = 130.76171875
SigRef 0.18143293986049897
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 45, 16%ile = 248.0, median stars = 288.0, 84%ile = 318.0, max stars = 364
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 172, 16%ile = 220.0, median stars = 245.0, 84%ile = 274.0, max stars = 350
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 124, 16%ile = 251.0, median stars = 283.0, 84%ile = 311.0, max stars = 363
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 138, 16%ile = 191.7, median stars = 218.0, 84%ile = 252.0, max stars = 315
Region for tile4

/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:44: RuntimeWarning: divide by zero encountered in true_divide
  df['reduced_chisq'] = df['chisq'].to_numpy() * n_passbands / (n_passbands - 4)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:49: RuntimeWarning: divide by zero encountered in power
  sdss_flux_sig = np.power(np.array(dat['sdss_dr14_starsweep.psfflux_ivar']), -0.5)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:51: RuntimeWarning: divide by zero encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log10(np.array(dat['sdss_dr14_starsweep.psfflux'])[:, ib]), 0.0, np.inf)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:56: RuntimeWarning: divide by zero encountered in log10
  df['ps.psfmagmean_'+b] = -2.5*np.log10(np.arr

Cut0 1190659 0.9977215936610194
Cut1 1159855 0.9719091520038077
Cut2 1165553 0.9766838336218701
Cut3 1060349 0.8885273567972596
Cut4 1193378 1.0
Cut5 724711 0.60727699019087
Cuts_All 672637 0.5636411933184624
Tile 4672, NumPixels=4096, NumStars=672637, Star/Pix ratio = 164.218017578125
SigRef 0.17803743721710752
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 251, 16%ile = 333.7, median stars = 372.5, 84%ile = 412.0, max stars = 483
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 259, 16%ile = 333.0, median stars = 362.0, 84%ile = 394.0, max stars = 451
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 166, 16%ile = 286.0, median stars = 325.0, 84%ile = 365.0, max stars = 419
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 197, 16%ile = 261.0, median stars = 314.0, 84%ile = 358.0, max stars = 424
Region for tile4673
Cu

...

Cut0 1107444 0.9973819089639222
Cut1 1070517 0.9641248578152314
Cut2 1086488 0.9785085977317083
Cut3 983064 0.8853632770178079
Cut4 1110101 0.9997748459721295
Cut5 767963 0.6916398508219473
Cuts_All 682291 0.614482267319073
Tile 4675, NumPixels=4096, NumStars=682291, Star/Pix ratio = 166.574951171875
SigRef 0.18167773165049175
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 265, 16%ile = 346.7, median stars = 388.0, 84%ile = 426.3, max stars = 516
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 224, 16%ile = 292.0, median stars = 331.0, 84%ile = 365.0, max stars = 464
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 123, 16%ile = 282.0, median stars = 323.0, 84%ile = 371.3, max stars = 461
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 184, 16%ile = 258.0, median stars = 289.0, 84%ile = 320.0, max stars = 386
Region 

/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:44: RuntimeWarning: divide by zero encountered in true_divide
  df['reduced_chisq'] = df['chisq'].to_numpy() * n_passbands / (n_passbands - 4)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:49: RuntimeWarning: divide by zero encountered in power
  sdss_flux_sig = np.power(np.array(dat['sdss_dr14_starsweep.psfflux_ivar']), -0.5)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:51: RuntimeWarning: divide by zero encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log10(np.array(dat['sdss_dr14_starsweep.psfflux'])[:, ib]), 0.0, np.inf)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:51: RuntimeWarning: invalid value encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log

Cut0 981214 0.9969589730615612
Cut1 946044 0.9612246204304582
Cut2 956910 0.9722649808424447
Cut3 860941 0.8747560218531264
Cut4 983912 0.999700266305767
Cut5 696446 0.7076214658095299
Cuts_All 612664 0.6224950645545093
Tile 4676, NumPixels=4096, NumStars=612664, Star/Pix ratio = 149.576171875
SigRef 0.18582348459428075
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 192, 16%ile = 302.0, median stars = 351.0, 84%ile = 391.0, max stars = 476
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 178, 16%ile = 282.0, median stars = 316.0, 84%ile = 347.0, max stars = 435
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 138, 16%ile = 280.0, median stars = 323.0, 84%ile = 361.0, max stars = 454
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 199, 16%ile = 259.0, median stars = 293.0, 84%ile = 333.0, max stars = 422
Region for til

/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:44: RuntimeWarning: divide by zero encountered in true_divide
  df['reduced_chisq'] = df['chisq'].to_numpy() * n_passbands / (n_passbands - 4)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:49: RuntimeWarning: divide by zero encountered in power
  sdss_flux_sig = np.power(np.array(dat['sdss_dr14_starsweep.psfflux_ivar']), -0.5)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:51: RuntimeWarning: divide by zero encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log10(np.array(dat['sdss_dr14_starsweep.psfflux'])[:, ib]), 0.0, np.inf)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:51: RuntimeWarning: invalid value encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log

Cut0 742119 0.9957359280930957
Cut1 725547 0.9735004971172566
Cut2 729612 0.9789546985966668
Cut3 662284 0.8886175578326493


In [35]:
missing_coords = np.flip(missing_coords)

In [48]:
looptilewise(missing_coords, recon_func_name, cuts_list, kwargs_dict, runname, save_tilewise=True, compile_pickle=False)

Looping over 620 tiles
Using dir  /n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/data/lsdraw/stars_edr3/{}.fits
Region for tile5575
Cut0 1336795 0.9954961227783143
Cut1 838582 0.6244825344437138
Cut2 1180716 0.8792658560978461
Cut3 1115231 0.8304999169672106
Cut4 1342843 1.0
Cut5 546034 0.4066253463733288
Cuts_All 258257 0.19232106806231258
Tile 5575, NumPixels=4096, NumStars=258257, Star/Pix ratio = 63.051025390625
SigRef 0.16498197314948415
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 20, 16%ile = 81.0, median stars = 123.0, 84%ile = 162.0, max stars = 253
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 22, 16%ile = 66.0, median stars = 93.0, 84%ile = 131.0, max stars = 202
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 47, 16%ile = 112.0, median stars = 137.0, 84%ile = 166.0, max stars = 215
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) o

KeyboardInterrupt: 

In [54]:
missing_tiles = []
for tile in numtiles:
    if not os.path.exists(runname+'_tmpdir/recon_{}.hdf5'.format(tile)):
        missing_tiles.append(tile)
print(len(missing_tiles))
missing_coords = []
for t32 in missing_tiles:
    missing_coords.append(get_smallpix_in_tilepix(outer_kwargs['Nsidetile'], t32, 2048))
missing_coords = np.hstack(missing_coords)
np.all(np.unique(get_largepix_for_smallpix(missing_coords, 2048, outer_kwargs['Nsidetile'])) == missing_tiles)

618


True

In [55]:
%%capture cap
looptilewise(missing_coords, recon_func_name, cuts_list, kwargs_dict, runname, save_tilewise=True, compile_pickle=False)

MemoryError: Unable to allocate 9.24 GiB for an array with shape (1024, 1211065) and data type float64

In [57]:
with open(runname+'_tmpdir/looptilewise_run2.txt', 'w') as f:
    f.write(cap.stdout)